### This function is an example of what we currently have running in our code base. Note: I believe the problem may be with PyCharm because this following code does recieve a speed boost in Jupyter, but not when run inside PyCharm. Thus the times I use will be from running the code in PyCharm to show how we might fix this

In [ ]:
from numba import njit, config
import numpy as np
import time

config.DISABLE_JIT = False


def test():
    timeStart = time.time()
    
    matrix1 = np.random.rand(1000, 1000)
    matrix2 = np.random.rand(1000, 1000)
    
    matrixMultiplied = matrixMuliplication(matrix1, matrix2)
    
    timeEnd = time.time()
    print(f"Total Time: {timeEnd - timeStart}")
    

@njit()
def matrixMuliplication(matrix1, matrix2):
    result = np.dot(matrix1, matrix2)
    return result
          
          
test()

Total Time: 0.8963, Let's run it again in PyCharm to see if there is a speedup:

Total Time: 0.9306, no speed up. I believe this is because PyCharm doesn't automatically cache the machine code JIT generates. Thus, it generates the machine level code every time it runs, resulting in the long run times. Let's change the code to fix this:

In [ ]:
from numba import njit, config
import numpy as np
import time

config.DISABLE_JIT = False


def test():
    timeStart = time.time()

    matrix1 = np.random.rand(1000, 1000)
    matrix2 = np.random.rand(1000, 1000)

    matrixMultiplied = matrixMuliplication(matrix1, matrix2)

    timeEnd = time.time()
    print(f"Total Time: {timeEnd - timeStart}")

# All that was changed was this line here, adding cache=True
@njit(cache=True)
def matrixMuliplication(matrix1, matrix2):
    result = np.dot(matrix1, matrix2)
    return result


test()

The first time it runs, there is no speed-up because it is generating the machine code. However the second time it runs with this new code we get a new speed: Total Time: 0.3127. Now that we see a speedup as expected, lets disable JIT and see performance with out it (We can use Jupyter for this since the timing results should be similar, we only used PyCharm at the start was because it seems Jupyter doesn't have this caching problem, however now that we figured out the caching problem we can use Jupyter for all examples since it will have correct results now)

In [15]:
from numba import njit, config
import numpy as np
import time

config.DISABLE_JIT = True


def test():
    timeStart = time.time()

    matrix1 = np.random.rand(1000, 1000)
    matrix2 = np.random.rand(1000, 1000)

    matrixMultiplied = matrixMuliplication(matrix1, matrix2)

    timeEnd = time.time()
    print(f"Total Time: {timeEnd - timeStart}")


@njit(cache=True)
def matrixMuliplication(matrix1, matrix2):
    result = np.dot(matrix1, matrix2)
    return result


test()

Total Time: 0.03860878944396973


We see here that JIT is still slower even though we figured out the caching problem and there is a speed up. This is because of the overhead that comes with JIT. Generating the machine code comes with extra overhead. Normally this extra time is negligible, because JIT increases performance by enough to offset it. However, look at the code we have above. All we are doing is multiplying two matrices together. This is not a complex operation, nor are we doing multiple calculations. For JIT to be useful, it has to overcome the extra overhead added by having enough calculations running. This is where our second slowdown problem has come from. That is why our test cases are slower in JIT (other than the obviously the caching problem). Let's take a look at a test case from UXarray:

In [ ]:
def test_calculate_face_area(self):
        """Test function for helper function calculate_face_area - only one face."""
        # Note: currently only testing one face, but this can be used to get area of multiple faces
        # Also note, this does not need face_nodes, assumes nodes are in counterclockwise orientation
        x = np.array([0.57735027, 0.57735027, -0.57735027])
        y = np.array([-5.77350269e-01, 5.77350269e-01, 5.77350269e-01])
        z = np.array([-0.57735027, -0.57735027, -0.57735027])

        area = ux.calculate_face_area(x, y, z, "gaussian", 5, "cartesian")

        nt.assert_almost_equal(area, constants.TRI_AREA, decimal=3)

This is finding the area of a single face by calling calculate_face_area(), which is a JIT enabled function. But the calculation isn't diffucult at all, the computer can calculate the area of one face extremely quickly. JIT however, adds extra overhead. So these test cases are not good benchmarks for JIT speedup at all. To get a good understand of the speed up, we should be using actually grid files, something like MPAS, to find the face areas of several thousand faces at the very least. At that point we should see a significant speed up boost as long as we implement the @njit(cache=True)

Here is a list of some of the more useful resources I read on this topic if you want more information:

https://santhalakshminarayana.github.io/blog/super-fast-python-numba

https://www.reddit.com/r/Python/comments/1wnt5h/numba_code_slower_than_pure_python_code/

https://stackoverflow.com/questions/59427775/numba-cache-true-has-no-effect

https://numba.discourse.group/t/why-is-numba-slow-on-a-high-performance-computing-cluster/575/3

https://stackoverflow.com/questions/65042223/why-numba-is-slower-than-pure-python-in-my-code

https://stackoverflow.com/questions/70455933/numba-is-not-enhancing-the-performance